In [12]:
import pandas as pd
import minsearch
import json

In [4]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [5]:
df = pd.read_csv("../data.csv")
documents = df.to_dict(orient='records')

In [6]:
prompt_template = """
    You emulate a user of our fitness assistant application.
    Formulate 5 questions this user might ask based on a provided exercise.
    Make the questions specific to this exercise.
    The record should contain the answer to the questions, and the questions should
    be complete and not too short. Use as fewer words as possible from the record. 
    
    The record:
    
    exercise_name: {exercise_name}
    type_of_activity: {type_of_activity}
    type_of_equipment: {type_of_equipment}
    body_part: {body_part}
    type: {type}
    muscle_groups_activated: {muscle_groups_activated}
    instructions: {instructions}
    
    Provide the output in parsable JSON without using code blocks:
    
    {{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [9]:
prompt = prompt_template.format(**documents[0])

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [14]:
questions = llm(prompt)

In [16]:
json.loads(questions)

{'questions': ['What is the starting position for performing push-ups?',
  'Which muscle groups are primarily activated during push-ups?',
  'Can push-ups be done without any equipment?',
  'What is the correct movement sequence for a push-up?',
  'Which body part is primarily targeted while doing push-ups?']}

In [19]:
index = minsearch.Index(
    text_fields = ['exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields = ['id']
)

In [21]:
index.fit(documents)

In [23]:
query = 'give me exercises for hamstrings' 

## RAG Flow

In [25]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

In [26]:
prompt_template = """
    You're a coach teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: {context}
""".strip()

entry_template = """
    exercise_name: {exercise_name}
    type_of_activity: {type_of_activity}
    type_of_equipment: {type_of_equipment}
    body_part: {body_part}
    type: {type}
    muscle_groups_activated: {muscle_groups_activated}
    instructions: {instructions}
""".strip()


def build_prompt(query, search_results):
    
    context = ""

    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [27]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [28]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
answer = rag(query)
print(answer)

Here are some exercises that target the hamstrings:

1. **Leg Curl**:
   - **Activity Type**: Strength
   - **Equipment**: Machine
   - **Instructions**: Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.

2. **Hamstring Stretch**:
   - **Activity Type**: Mobility
   - **Equipment**: Bodyweight
   - **Instructions**: Stand upright and place one heel on a bench or step. Lean forward slightly to feel a stretch in the hamstring of the elevated leg.

3. **Machine Leg Curl**:
   - **Activity Type**: Strength
   - **Equipment**: Machine
   - **Instructions**: Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.

4. **Seated Leg Curl**:
   - **Activity Type**: Strength
   - **Equipment**: Machine
   - **Instructions**: Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.

5. **Prone Leg Curl**:
   - **Activity Type**: Strength